#### Create more interactive maps showing classification outcomes using Folium
- overlay open street maps using cartoDB baselayers  
- show where correct/wrong-ly classified images are located  
- save as html file 


- Ref: http://localhost:8888/notebooks/load_HoldoutPredictions_plotImageClassPredictions_v0.ipynb  
- previous static version: http://localhost:8888/notebooks/explore_imagesNgeojson_v1.ipynb

#### General REQs

In [108]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import numpy as np
import pandas as pd

import geopandas as gpd
import gdal

#### Load geojson info | data

In [54]:
df_joined = pd.read_csv('./building_set/combined_geojsonInfo.csv', index_col=0)

In [55]:
Gdf = gpd.GeoDataFrame(df_joined) #not sure if this converts it properly...

In [62]:
Gdf.head()

,index,class_name,feature_id,geometry,x,y,xy
0,0,Buildings,298021,"POLYGON ((14.20433772170354 12.36790308324247,...",14.204910,12.368465,"[14.204909841113661, 12.368464982853263]"
1,1,Buildings,624999,"POLYGON ((13.57843908703417 11.33400779938784,...",13.579011,11.334570,"[13.57901120644429, 11.334569698998633]"
2,2,Buildings,297460,"POLYGON ((13.95718213653246 11.35873138226263,...",13.957754,11.359293,"[13.957754255942577, 11.359293281873416]"
3,3,Buildings,57828,"POLYGON ((13.69629568551853 11.5149394740624, ...",13.696868,11.515501,"[13.696867804928653, 11.515501373673192]"
4,4,Buildings,162112,"POLYGON ((14.28100172265939 11.33288400016626,...",14.281574,11.333446,"[14.281573842069509, 11.33344589977705]"


In [63]:
Gdf_B = Gdf[Gdf.class_name=='Buildings']
Gdf_NB = Gdf[Gdf.class_name=='No Buildings']

In [65]:
Gdf_B.shape, Gdf_NB.shape

((2000, 7), (20000, 7))

### Refs for Folium & Carto Mapping
- https://folium.readthedocs.io/en/latest/index.html || https://github.com/python-visualization/folium
- https://gist.github.com/wrobstory/5558986  
- https://python-visualization.github.io/folium/module/features.html  
- https://ocefpaf.github.io/python4oceanographers/blog/2015/12/14/geopandas_folium/


- https://carto.com/location-data-services/basemaps/
- https://carto.com/docs/carto-editor/maps#including-an-external-basemap


In [ ]:
# import folium
# from matplotlib.colors import Normalize, rgb2hex 
# import matplotlib.cm as cm


#### Try Basic mapping... of approx. mean lat/long coords 

In [129]:
import folium
from matplotlib.colors import Normalize, rgb2hex 
import matplotlib.cm as cm


# norm = Normalize(Var.min(), Var.max())

# Get dark tileset from CartoBD (https://cartodb.com/basemaps)
tileset = r'http://{s}.basemaps.cartocdn.com/light_all/{z}/{x}/{y}@2x.png'

test_map = folium.Map(location = [14, 12], width = 980, height = 800,
                               tiles = tileset,
                               attr = '&copy; <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors, &copy; <a href="http://cartodb.com/attributions">CartoDB</a>',
                               zoom_start = 3)

test_map.add_child(folium.CircleMarker(location = [12,14],
                                        popup = 'test',
                                        radius = 5,
                                        color = 'black',#'#f44242',
                                        fill_color = 'black',#'#f44242',# line_color = '#f44242'
                                      )
                  )

test_map.save(outfile = './Folium_maps/test_map.html')

del test_map

In [130]:
from IPython.display import IFrame
IFrame('./Folium_maps/test_map.html', width = 980, height = 800)

#### Plot all B | NB image centroid loc 

In [123]:
import folium
from matplotlib.colors import Normalize, rgb2hex 
import matplotlib.cm as cm


# norm = Normalize(Var.min(), Var.max())
# color = rgb2hex(cm.nipy_spectral(norm(row['Var'])))

# Get dark tileset from CartoBD (https://cartodb.com/basemaps)
tileset = r'http://{s}.basemaps.cartocdn.com/light_all/{z}/{x}/{y}@2x.png'

test_map = folium.Map(location = [14, 12], width = 980, height = 800,
                               tiles = tileset,
                               attr = '&copy; <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors, &copy; <a href="http://cartodb.com/attributions">CartoDB</a>',
                               zoom_start = 3)


for index, row in Gdf_NB.iterrows():
    test_map.add_child(folium.CircleMarker(location = [row['y'],row['x']],
                                            popup = 'No Building',
                                            radius = 5,
                                            color = '#606060', #'#bababa', #'#158c3e',#
                                            fill_color = '#606060', #'#bababa', #'black', #'#158c3e',# line_color = '#f44242'
                                          )
                      )
    
for index, row in Gdf_B.iterrows():
    test_map.add_child(folium.CircleMarker(location = [row['y'],row['x']],
                                            popup = 'Building',
                                            radius = 5,
                                            color = '#f4a742', #'#f44242',
                                            fill_color = '#f4a742', #'#f44242',# line_color = '#f44242'
                                          )
                      )


# test_map.save(outfile = './Folium_maps/map_B_NB_v1.html') ##takes forever to build... don't overwrite...
test_map.save(outfile = './Folium_maps/test_map1.html')

del test_map

In [131]:
from IPython.display import IFrame
IFrame('./Folium_maps/map_B_NB_v4.html', width = 980, height = 800)
# IFrame('./Folium_maps/test_map1.html', width = 980, height = 800)

#### Plot Holdout Classification Outcomes for B | NB images

In [98]:
df_HOpred = pd.read_csv('./building_set/HoldoutPredict_CombinedGeoInfo.csv')

In [101]:
Gdf_HOpred = gpd.GeoDataFrame(df_HOpred)

In [103]:
df_HOpred.head()

,Unnamed: 0,filepath,feature_id,class_name,Blabel,augment,YHoldPredProb_nb,YHoldPredProb_b,YHold_test,YHold_predict,predict_diff,geometry,x,y,xy
0,0,building_set/buildings/760690.tif,760690,Buildings,1,0,0.014134,0.985866,1,1,0,"POLYGON ((13.85420064271117 11.37558837058634,...",13.854773,11.376150,"[13.854772762121291, 11.376150270197135]"
1,1,building_set/buildings/841079.tif,841079,Buildings,1,0,0.031087,0.968913,1,1,0,"POLYGON ((14.61168674170774 12.28137054318073,...",14.612259,12.281932,"[14.612258861117859, 12.28193244279152]"
2,2,building_set/buildings/615983.tif,615983,Buildings,1,0,0.005324,0.994676,1,1,0,"POLYGON ((14.19518381114165 12.36790308324247,...",14.195756,12.368465,"[14.19575593055177, 12.368464982853263]"
3,3,building_set/buildings/228006.tif,228006,Buildings,1,0,0.758569,0.241431,1,0,1,"POLYGON ((14.18831837822023 12.33531290581662,...",14.188890,12.335875,"[14.188890497630352, 12.335874805427412]"
4,4,building_set/buildings/782943.tif,782943,Buildings,1,0,0.071391,0.928609,1,1,0,"POLYGON ((14.10364470552273 11.64642398298739,...",14.104217,11.646986,"[14.10421682493285, 11.646985882598184]"


In [ ]:
B_predDF = df_HOpred[df_HOpred.Blabel==1]
NB_predDF = df_HOpred[df_HOpred.Blabel==0]

B_pNB_DF = df_HOpred[df_HOpred.predict_diff==1]
NB_pB_DF = df_HOpred[df_HOpred.predict_diff==-1]


In [ ]:
import folium
from matplotlib.colors import Normalize, rgb2hex 
import matplotlib.cm as cm


# norm = Normalize(Var.min(), Var.max())
# color = rgb2hex(cm.nipy_spectral(norm(row['Var'])))

# Get dark tileset from CartoBD (https://cartodb.com/basemaps)
tileset = r'http://{s}.basemaps.cartocdn.com/light_all/{z}/{x}/{y}@2x.png'

test_map = folium.Map(location = [14, 12], width = 1600, height = 1000,
                               tiles = tileset,
                               attr = '&copy; <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors, &copy; <a href="http://cartodb.com/attributions">CartoDB</a>',
                               zoom_start = 3)


for index, row in NB_predDF.iterrows():
    test_map.add_child(folium.CircleMarker(location = [row['y'],row['x']],
                                            popup = 'No Building',
                                            radius = 5,
                                            color = '#606060',#'#158c3e',#
                                            fill_color = '#606060', #'black', #'#158c3e',# line_color = '#f44242'
                                          )
                      )
    
for index, row in B_predDF.iterrows():
    test_map.add_child(folium.CircleMarker(location = [row['y'],row['x']],
                                            popup = 'Building',
                                            radius = 5,
                                            color = '#f4a742', #'#f44242',
                                            fill_color = '#f4a742', #'#f44242',# line_color = '#f44242'
                                          )
                      )


for index, row in NB_pB_DF.iterrows():
    test_map.add_child(folium.CircleMarker(location = [row['y'],row['x']],
                                            popup = 'No Building | predicted Building',
                                            radius = 5,
                                            color = 'black' ,#'#bababa', #'#
                                            fill_color = 'black',#'#bababa', #'black', #'#158c3e',# line_color = '#f44242'
                                          )
                      )
    
for index, row in B_pNB_DF.iterrows():
    test_map.add_child(folium.CircleMarker(location = [row['y'],row['x']],
                                            popup = 'Building | Predicted No Building',
                                            radius = 5,
                                            color = '#6d0c0c', #'#f44242',
                                            fill_color = '#6d0c0c', #'#f44242',# line_color = '#f44242'
                                          )
                      )
    
    
# test_map.save(outfile = './Folium_maps/map_predict_v0.html') ##takes forever to build... don't overwrite...
test_map.save(outfile = './Folium_maps/test_map2.html')

del test_map

In [125]:
from IPython.display import IFrame
IFrame('./Folium_maps/map_predict_v1.html', width = 980, height = 800)
# IFrame('./Folium_maps/test_map2.html', width = 980, height = 800)